# 1. Save_snapshot

In [ ]:
from jetbot import Camera, bgr8_to_jpeg
import traitlets
import os
import uuid
import time
import ipywidgets.widgets as widgets

# 폴더 경로 설정 - 이미지를 저장할 폴더
save_folder = './'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Camera 인스턴스 생성 및 이미지 디스플레이 위젯 설정
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# 이미지 저장 함수 정의
def save_snapshot():
    global save_folder
    img_path = os.path.join(save_folder, 'quest.jpg')  # 이미지 이름을 무작위로 생성하여 저장
    with open(img_path, 'wb') as f:
        f.write(image.value)
        
save_snapshot()

# 2. Buzzer

In [ ]:
import RPi.GPIO as GPIO

BEEP_pin = 6 # buzzer pin number

GPIO.setmode(GPIO.BCM)
GPIO.setup(BEEP_pin, GPIO.OUT, initial = GPIO.LOW) # starting option

import time

def Beep(beeptime):
    GPIO.output(BEEP_pin, GPIO.HIGH) # buzzer on
    time.sleep(beeptime) # sound time
    GPIO.output(BEEP_pin, GPIO.LOW) # buzzer off

# 3. Image Classifier

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

def ImageClassifier(image_path, url) 
    # 크롬 드라이버 경로
    chrome_driver_path = "/usr/bin/chromedriver" # 'https://ideal-man58.github.io/ai_testing/public/index.html'

    # 크롬 옵션 설정
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # 브라우저를 실제로 열지 않고 실행 (화면이 필요 없을 때 유용)

    # 웹 드라이버 초기화
    driver = webdriver.Chrome(options=chrome_options)

    # 웹 페이지 열기
    driver.get(url)

    # 이미지 업로드
    image_input.send_keys(image_path)

    # 잠시 대기하여 JavaScript가 실행되도록 기다림
    time.sleep(1)

    # 결과 파싱 (결과 요소의 ID를 실제 웹 페이지에서 확인하여 입력)
    result_element = driver.find_element("css selector", "#resultDisplay")
    result = result_element.text

    # 웹 드라이버 종료
    driver.quit()

    result = result.split()
    result = [result[1], result[3]]
    print('Classification Result:', result)
    return result
 

# 4. Avoid Obstacle

In [ ]:
def update(image_path):
    obstacle = 'https://ideal-man58.github.io/ai_testing/public/index.html'
    endpoint = 'https://kkamny.github.io/ai_testing/public/index.html'
    if ('blocked' in ImageClassifier(image_path, obstacle)) :
        if('other' in ImageClassifier(image_path, endpoint)):
            robot.stop()
            Beep(1)

            robot.left(1)
            time.sleep(0.27)
            robot.stop()
            robot.forward(2)
            time.sleep(0.5)
            robot.stop()
            robot.right(1)
            time.sleep(0.3)
            robot.stop()
            return 1
            
        else:
            robot.stop()
            return 0
        
    else :
            robot.stop()
            robot.forward(2)
            time.sleep(0.5)
            robot.stop()
            return 1

# 5. Start

In [ ]:
while(1) :
    save_snapshot()
    state = update('./quest.jpg')
    if state == 0 :
        break